# Bitcoin Analysis Stack - Getting Started

This notebook demonstrates basic usage of the optimized Bitcoin Analysis Stack with shared blockchain storage.

## 1. Connect to Bitcoin Core

In [ ]:
import os
from bitcoinrpc.authproxy import AuthServiceProxy

# Connect to Bitcoin Core
btc_host = os.getenv('BITCOIN_RPC_HOST', 'bitcoin')
btc_port = os.getenv('BITCOIN_RPC_PORT', '8332')
btc_user = os.getenv('BITCOIN_RPC_USER', 'btcuser')
btc_pass = os.getenv('BITCOIN_RPC_PASSWORD', 'btcpass')

btc = AuthServiceProxy(f"http://{btc_user}:{btc_pass}@{btc_host}:{btc_port}")

# Get blockchain info
info = btc.getblockchaininfo()
print(f"Chain: {info['chain']}")
print(f"Blocks: {info['blocks']:,}")
print(f"Size on disk: {info['size_on_disk'] / 1e9:.2f} GB")

## 2. Connect to Neo4j Graph Database

In [ ]:
from neo4j import GraphDatabase

# Connect to Neo4j
neo4j_uri = os.getenv('NEO4J_URI', 'bolt://neo4j:7687')
neo4j_user = os.getenv('NEO4J_USER', 'neo4j')
neo4j_pass = os.getenv('NEO4J_PASSWORD', 'bitcoin123')

driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_pass))

# Test connection
with driver.session() as session:
    result = session.run("""
        MATCH (b:Block)
        RETURN count(b) as block_count
    """)
    record = result.single()
    print(f"Blocks in Neo4j: {record['block_count']:,}")

## 3. Query GraphQL API

In [ ]:
import requests

graphql_endpoint = os.getenv('GRAPHQL_ENDPOINT', 'http://graphql:8000/graphql')

# Query blockchain info
query = """
query {
  blockchainInfo {
    blocks
    chain
    difficulty
    sizeOnDisk
  }
}
"""

response = requests.post(graphql_endpoint, json={'query': query})
data = response.json()
print(data)

## 4. Address Analysis Example

In [ ]:
# Analyze Satoshi's address (Genesis block coinbase)
address = "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa"

with driver.session() as session:
    result = session.run("""
        MATCH (a:Address {address: $address})
        OPTIONAL MATCH (a)<-[r:OUTPUTS_TO]-(:Transaction)
        RETURN a.address as address,
               a.first_seen as first_seen,
               sum(r.value) as total_received,
               count(r) as tx_count
    """, address=address)
    
    record = result.single()
    if record and record['address']:
        print(f"Address: {record['address']}")
        print(f"First Seen: {record['first_seen']}")
        print(f"Total Received: {record['total_received'] or 0:.8f} BTC")
        print(f"Transaction Count: {record['tx_count'] or 0}")
    else:
        print("Address not found in database (may not be imported yet)")

## 5. Transaction Flow Analysis

In [ ]:
# Find most active addresses
with driver.session() as session:
    result = session.run("""
        MATCH (a:Address)<-[r:OUTPUTS_TO]-(t:Transaction)
        RETURN a.address as address,
               count(t) as tx_count,
               sum(r.value) as total_received
        ORDER BY tx_count DESC
        LIMIT 10
    """)
    
    print("Top 10 Most Active Addresses:")
    print("-" * 80)
    for i, record in enumerate(result, 1):
        print(f"{i}. {record['address'][:20]}... | "
              f"Txs: {record['tx_count']:,} | "
              f"Received: {record['total_received']:.2f} BTC")

## 6. Check Storage Optimization

This stack uses a single shared blockchain volume, reducing storage requirements significantly.

In [ ]:
import subprocess

# Check docker volumes
result = subprocess.run(['docker', 'volume', 'ls'], capture_output=True, text=True)
print("Docker Volumes:")
print(result.stdout)

# Note: All services share bitcoin_data volume (read-only for most)
# Only Bitcoin Core writes to it, others read
# This saves ~600GB compared to having separate copies

## 7. Redis Cache Status

In [ ]:
import redis

# Connect to Redis
redis_host = os.getenv('REDIS_HOST', 'redis')
redis_port = int(os.getenv('REDIS_PORT', '6379'))

r = redis.Redis(host=redis_host, port=redis_port, decode_responses=True)

# Get cache info
info = r.info('memory')
print(f"Redis Memory Used: {info['used_memory_human']}")
print(f"Redis Keys: {r.dbsize()}")

# The GraphQL API and importer use Redis for caching
# This significantly reduces RPC calls to Bitcoin Core

## Next Steps

- Explore address clustering algorithms
- Visualize transaction graphs with NetworkX or Pyvis
- Create custom analysis scripts
- Query the GraphQL API for complex patterns